In [1]:
# Grupuojami Zive ekg įrašai į to paties paciento grupes
# Galima keisti if timediff > timedelta(hours=24) hours = 24, gausime kitą suskirstymą.
# Sugrupuotų įrašų sąrašas pacientai.json įrašomas į aplanką rec_dir_target.
# Testinėje dalyje nuskaito failą pacientai.json ir išveda apibendrintą informaciją.


import pandas as pd
import numpy as np
import json, sys
from pathlib import Path
import shutil
from datetime import datetime, timedelta

from collect_list import collect_list 

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)


# Vietinės talpyklos aplankas
db_folder = 'DUOMENU_TVARKYMAS_2023'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais ir anotacijomis (.json), iš kurio skaitomi atrinkti įrašai
rec_dir_source = Path(db_path, 'DUOMENYS_IS_SERVERIO_2023_03_26')

# Nuoroda į aplanką su EKG įrašais ir anotacijomis (.json), į kurį perrašomi atrinkti įrašai
rec_dir_target = Path(db_path, 'ATRINKTI_DUOMENYS')

# Sukuriame įrašų aplanke rec_dir_source sąrašą

df_list  = collect_list(rec_dir_target)
df_list['recNr'] = 0
# df_list['recNr'] = df_list['recNr'].astype(int)


# print(filenames)
# print(df_list.head(3))

# convert recorded_at to datetime
df_list = df_list.reset_index(drop=True)
# print(df_list.head(3))

df_list['recorded_at'] = pd.to_datetime(df_list['recorded_at'])

# group by userId and sort by recorded_at
grouped = df_list.sort_values('recorded_at').groupby('userId')

result = []
for user, group_df in grouped:
    print("user:", user)
    subgroups = []
    recNr = 0
    row_0 = group_df.iloc[0].copy()
    row_0['recNr'] = recNr
    current_subgroup = [row_0]
    for i in range(1, len(group_df)):
        timediff = group_df.iloc[i]['recorded_at'] - group_df.iloc[i-1]['recorded_at']
        print(i, timediff)
        row_i = group_df.iloc[i].copy()
        if timediff > timedelta(hours=24):
            subgroups.append(current_subgroup)
            recNr += 1
            row_i['recNr'] = recNr
            current_subgroup = [row_i]
        else:
            current_subgroup.append(row_i)
    subgroups.append(current_subgroup)
    result.append(subgroups)
print('len(result):',len(result))


# print('1',result[0])
# print('2',result[0][0])
# print('3',result[0][0][0])
# print('4',result[0][0][0]['file_name'])

json_object = []
for i in range(len(result)):
    subgroups = result[i]
    for j in range(len(subgroups)):
        subgroup = subgroups[j]
        userId = subgroup[0]['userId']
        recNr = subgroup[0]['recNr']
        file_names = []
        recorded_ats = []
        for k in range(len(subgroup)):
            file_names.append(subgroup[k]['file_name'])
            recorded_ats.append(subgroup[k]['recorded_at'].strftime('%Y-%m-%d %H:%M:%S'))
        print(f"userId: {userId}, 'recNr': {recNr} file_names: {file_names}, recorded_ats: {recorded_ats}")
        json_object.append({'userId': userId, 'recNr': int(recNr), 'file_names': file_names, 'recorded_ats': recorded_ats})

file_name = 'pacientai.json'
file_path = Path(rec_dir_target, file_name)

# Open a file in write mode
# Write the list of dictionaries to the file in JSON format
with open(file_path, "w") as outfile:
    json.dump(json_object, outfile)    


    

OS in my system :  linux
user: 100
1 0 days 00:16:41
2 0 days 00:16:41
user: 101
1 0 days 00:16:41
2 0 days 00:16:41
user: 103
1 0 days 00:16:41
2 0 days 00:16:41
user: 105
1 0 days 00:16:41
2 0 days 00:16:41
user: 106
1 0 days 00:16:41
2 0 days 00:16:41
user: 108
1 0 days 00:16:41
2 0 days 00:16:41
user: 109
1 0 days 00:16:41
2 0 days 00:16:41
user: 111
1 0 days 00:16:41
2 0 days 00:16:41
user: 112
1 0 days 00:16:41
2 0 days 00:16:41
user: 113
1 0 days 00:16:41
2 0 days 00:16:41
user: 114
1 0 days 00:16:41
2 0 days 00:16:41
user: 115
1 0 days 00:16:41
2 0 days 00:16:41
user: 116
1 0 days 00:16:41
2 0 days 00:16:41
user: 117
1 0 days 00:16:41
2 0 days 00:16:41
user: 118
1 0 days 00:16:41
2 0 days 00:16:41
user: 119
1 0 days 00:16:41
2 0 days 00:16:41
user: 121
1 0 days 00:16:41
2 0 days 00:16:41
user: 122
1 0 days 00:16:41
2 0 days 00:16:41
user: 123
1 0 days 00:16:41
2 0 days 00:16:41
user: 124
1 0 days 00:16:41
2 0 days 00:16:41
user: 200
1 0 days 00:16:41
2 0 days 00:16:41
user: 201

In [2]:
# Nuskaito failą pacientai.json ir išveda apibendrintą informaciją

file_name = 'pacientai.json'
file_path = Path(rec_dir_target, file_name)

# open file pacientai.json and read json
        
with open(file_path) as json_file:
    pacients = json.load(json_file)
    
# print(pacients)

print("\nPacientų įrašų skaičius:", len(pacients))

dict_list = []
for idx, record in enumerate(pacients):
    dict = {'nr':idx, 'userId': record['userId'], 'recNr': record['recNr'], 'number of parts': len(record['file_names'])}
    dict_list.append(dict)
for idx in range(len(dict_list)):    
    print(dict_list[idx])


Pacientų įrašų skaičius: 93
{'nr': 0, 'userId': '100', 'recNr': 0, 'number of parts': 3}
{'nr': 1, 'userId': '101', 'recNr': 0, 'number of parts': 3}
{'nr': 2, 'userId': '103', 'recNr': 0, 'number of parts': 3}
{'nr': 3, 'userId': '105', 'recNr': 0, 'number of parts': 3}
{'nr': 4, 'userId': '106', 'recNr': 0, 'number of parts': 3}
{'nr': 5, 'userId': '108', 'recNr': 0, 'number of parts': 3}
{'nr': 6, 'userId': '109', 'recNr': 0, 'number of parts': 3}
{'nr': 7, 'userId': '111', 'recNr': 0, 'number of parts': 3}
{'nr': 8, 'userId': '112', 'recNr': 0, 'number of parts': 3}
{'nr': 9, 'userId': '113', 'recNr': 0, 'number of parts': 3}
{'nr': 10, 'userId': '114', 'recNr': 0, 'number of parts': 3}
{'nr': 11, 'userId': '115', 'recNr': 0, 'number of parts': 3}
{'nr': 12, 'userId': '116', 'recNr': 0, 'number of parts': 3}
{'nr': 13, 'userId': '117', 'recNr': 0, 'number of parts': 3}
{'nr': 14, 'userId': '118', 'recNr': 0, 'number of parts': 3}
{'nr': 15, 'userId': '119', 'recNr': 0, 'number of 